In [1]:
from auth_TDX import get_api_return_df
import pandas  as pd

### 拿到車站資料對應的車站號碼

In [2]:
#get basic information
api_use = '/v3/Rail/TRA/Station'
train_basic_info = get_api_return_df(api_use)
train_basic_info.head() #check header

c:\Users\Dell\Documents\TW_railway\auth_TDX.py:60: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data_response_df = pd.read_json(data_response.text)


,UpdateTime,UpdateInterval,SrcUpdateTime,SrcUpdateInterval,AuthorityCode,Stations
0,2024-07-16T06:55:17+08:00,14400,2024-07-16T04:00:00+08:00,-1,TRA,"{'StationUID': 'TRA-0900', 'StationID': '0900'..."
1,2024-07-16T06:55:17+08:00,14400,2024-07-16T04:00:00+08:00,-1,TRA,"{'StationUID': 'TRA-0910', 'StationID': '0910'..."
2,2024-07-16T06:55:17+08:00,14400,2024-07-16T04:00:00+08:00,-1,TRA,"{'StationUID': 'TRA-0920', 'StationID': '0920'..."
3,2024-07-16T06:55:17+08:00,14400,2024-07-16T04:00:00+08:00,-1,TRA,"{'StationUID': 'TRA-0930', 'StationID': '0930'..."
4,2024-07-16T06:55:17+08:00,14400,2024-07-16T04:00:00+08:00,-1,TRA,"{'StationUID': 'TRA-0940', 'StationID': '0940'..."


In [4]:
train_basic_info['Stations'][0] #check what's the data inside Ststions looks like

{'StationUID': 'TRA-0900',
 'StationID': '0900',
 'StationName': {'Zh_tw': '基隆', 'En': 'Keelung'},
 'StationPosition': {'PositionLon': 121.73997, 'PositionLat': 25.13411},
 'StationAddress': '203001基隆市中山區中山一路16之1號',
 'StationPhone': '02-24263743',
 'StationClass': '1',
 'StationURL': 'https://www.railway.gov.tw/tra-tip-web/tip/tip00H/tipH41/viewStaInfo/0900'}

In [3]:
def gen_train_hash_from_df(data)->dict:
    """
    Args:
        data (_type_): API回傳的車站資料

    Raises:
        ValueError: 如果遇到異常，就反應錯誤

    Returns:
        dict: 做好的hash map
    """
    data = {}
    try:
        for i, row in train_basic_info.iterrows():
            station_name = row['Stations']['StationName']['Zh_tw']
            station_id = row['Stations']['StationID']
            data[station_name] = station_id
    except:
        raise ValueError('Get unexpected data type.')
    return data

In [4]:
train_hash = gen_train_hash_from_df(train_basic_info)

### 拿到給定啟始結束車站以及日期的發車資訊

In [17]:
'臺南'.replace('臺', '台') #some input might be incorrect, add a secure

'台南'

In [5]:
def get_train_num_from_train_name(train_name:str, hash:dict=train_hash)->str:
    train_name_new = train_name.replace('台','臺')
    return hash[train_name_new]

def get_today_format()->str:
    import datetime
    current_date = datetime.date.today()
    return current_date.strftime("%Y-%m-%d")

def get_train_list_from_ini_station_fin_station_date(ini_station:str, fin_station:str, train_date:str=get_today_format())->pd.DataFrame:
    """_summary_

    Args:
        ini_station (str): input as a two word station
        fin_station (str): input as a two word station
        train_date (str, optional): _description_. Defaults to get_today_format(), format yyyy-MM-dd

    Returns:
        pd.DateFrame: Data from API
    """
    OriginStationID = get_train_num_from_train_name(ini_station)
    DestinationStationID = get_train_num_from_train_name(fin_station)
    test = f'/v3/Rail/TRA/DailyTrainTimetable/OD/Inclusive/{OriginStationID}/to/{DestinationStationID}/{train_date}'
    train_time_table = get_api_return_df(test)
    return train_time_table

In [6]:
get_train_data = get_train_list_from_ini_station_fin_station_date('后里','豐原')

c:\Users\Dell\Documents\TW_railway\auth_TDX.py:60: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data_response_df = pd.read_json(data_response.text)


In [37]:
temp = get_train_data['TrainTimetables'][0] #check what's inside
temp

{'TrainInfo': {'TrainNo': '177', 'Direction': 1, 'TrainTypeID': '110A', 'TrainTypeCode': '3', 'TrainTypeName': {'Zh_tw': '自強(推拉式自強號且有自行車車廂)', 'En': 'Tze-Chiang Limited Express'}, 'TripHeadSign': '往斗南', 'StartingStationID': '7000', 'StartingStationName': {'Zh_tw': '花蓮', 'En': 'Hualien'}, 'EndingStationID': '3480', 'EndingStationName': {'Zh_tw': '斗南', 'En': 'Dounan'}, 'TripLine': 1, 'WheelChairFlag': 1, 'PackageServiceFlag': 0, 'DiningFlag': 0, 'BreastFeedFlag': 1, 'BikeFlag': 1, 'CarFlag': 0, 'DailyFlag': 1, 'ExtraTrainFlag': 0, 'SuspendedFlag': 0, 'Note': '每日行駛。'}, 'StopTimes': [{'StopSequence': 25, 'StationID': '3220', 'StationName': {'Zh_tw': '后里', 'En': 'Houli'}, 'ArrivalTime': '19:56', 'DepartureTime': '19:57', 'SuspendedFlag': 0}, {'StopSequence': 26, 'StationID': '3230', 'StationName': {'Zh_tw': '豐原', 'En': 'Fengyuan'}, 'ArrivalTime': '20:03', 'DepartureTime': '20:05', 'SuspendedFlag': 0}, {'StopSequence': 27, 'StationID': '3250', 'StationName': {'Zh_tw': '潭子', 'En': 'Tanzi'}, 'A

In [46]:
temp['TrainInfo']['TrainNo'] # this is train number
temp['TrainInfo']['StartingStationID'] #this is the answer of whether train start is station start

temp['StopTimes'][0]['StationID'] #here is the stop station ID
temp['StopTimes'][0]['ArrivalTime'] #here is the stop station time

'19:56'

In [7]:
def get_arrivetime_by_strain_info(train_info:list, train_list:list)->dict:
    """_summary_

    Args:
        train_info (list): this should be train API return dataframe ['TrainTimetables'][i]['StopTimes'], each element in list is a dict
        train_list (list): list of train id to get time

    Returns:
        dict: train start time by input train list sequence, key is train id and value is time
    """
    res = {}
    for i in range(len(train_info)):
        station_id = train_info[i]['StationID']
        if station_id == train_list[0]:
            res['Start_Station'] = train_info[i]['ArrivalTime']
        elif station_id == train_list[1]:
            res['Arrive_Station'] = train_info[i]['ArrivalTime']
    return res

def get_train_sum_info_from_train_dict(train_info:dict, ini_station:str, fin_station:str):
    """_summary_

    Args:
        train_info (dict): data from API
        ini_station (str): in chinese
        fin_station (str): in chinese
    """
    train_num = train_info['TrainInfo']['TrainNo']
    train_start_loc = train_info['TrainInfo']['StartingStationID']
    OriginStationID = get_train_num_from_train_name(ini_station)
    DestinationStationID = get_train_num_from_train_name(fin_station)
    time_hash = get_arrivetime_by_strain_info(train_info['StopTimes'], [OriginStationID, DestinationStationID])
    time_hash['Train_ID'] = train_num
    time_hash['Is_init_station'] = (train_start_loc==OriginStationID)
    return time_hash

def gen_train_api_info_to_df(data:pd.DataFrame, ini_station:str, fin_station:str)->pd.DataFrame:
    summary_train_status = []
    for i in data['TrainTimetables']:
        summary_train_status.append(get_train_sum_info_from_train_dict(i, ini_station,fin_station))
    return pd.DataFrame(summary_train_status)

In [9]:
gen_train_api_info_to_df(get_train_data, '后里','豐原')

,Start_Station,Arrive_Station,Train_ID,Is_init_station
0,22:59,23:06,155,False
1,19:56,20:03,177,False
2,07:35,07:41,2005,False
3,21:05,21:12,2037,False
4,06:11,06:17,2103,False
5,06:58,07:06,2113,False
6,07:58,08:04,2123,False
7,08:48,08:54,2133,False
8,09:51,09:57,2143,False
9,10:55,11:01,2153,False
